In [ ]:
import chess
import chess.engine

In [ ]:
import numpy
import matplotlib.pyplot as plt

In [ ]:
import pandas as  pd

# Future prospect

In [ ]:
df = pd.read_csv('chessData.csv')

In [ ]:
df.head()

In [ ]:
df2=df["FEN"]
df2

In [ ]:
df3=df["Evaluation"]
df3

In [ ]:
df

In [ ]:
df4

In [ ]:
for row in df2:
    board3d = split_dims(board=chess.Board(row))
    

## Converts board into a 3d matrix

In [ ]:
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}

def square_to_index(square):
    letter = chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]

def split_dims(board):
  # this is the 3d matrix
    board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # here we add the pieces's view on the matrix
    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = numpy.unravel_index(square, (8, 8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = 1
        for square in board.pieces(piece, chess.BLACK):
            idx = numpy.unravel_index(square, (8, 8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
    aux = board.turn
    board.turn = chess.WHITE
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[12][i][j] = 1
    board.turn = chess.BLACK
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[13][i][j] = 1
    board.turn = aux

    return board3d

## Visualization of 3d matrix

In [ ]:
board=chess.Board('2K5/q7/P7/2k5/8/8/8/8 w - - 4 57')
board

In [ ]:
board3d = split_dims(board)
board3d

## Reading pgn file

In [ ]:
board = chess.Board()

In [ ]:
games = []
with open('F:\\chess\\ChessAI-master\\filtered_games.pgn', 'r') as f:
    lines = []
    for line in f:
        if line == "\n":
            continue
        lines.append(line.strip())
        if len(lines) == 16:
            games.append(lines)
            lines = []


In [ ]:
import chess.pgn
from io import StringIO

game_pgn = StringIO("\n".join(games[0]))

game = chess.pgn.read_game(game_pgn)
print(game)

## decode fen

In [ ]:
def parse_fen(fen_str):
    
    board, turn, castling, enp, halfmove, fullmove = fen_str.split(" ")
    for i in range(1, 9):
        board = board.replace(str(i), '.' * i)
    return board.replace("/", ""), turn, castling, enp, halfmove, fullmove


In [ ]:
import numpy as np

def board_to_nums(board_str):
    pieces = ['.', 'P', 'N', 'B', 'R', 'Q', 'K', 'p', 'n', 'b', 'r', 'q', 'k']
    board_array = np.zeros(64)
    for i, element in enumerate(board_str):
        board_array[i] = pieces.index(element)
    return board_array

def castling_to_nums(castling_str):
    cnums = np.zeros(4)
    if castling_str == "-":
        return cnums
    else:
        if "K" in castling_str:
            cnums[0] = 1
        if "Q" in castling_str:
            cnums[1] = 1
        if "k" in castling_str:
            cnums[2] = 1
        if "q" in castling_str:
            cnums[3] = 1
    return cnums

In [ ]:
def create_row(game_state):
    try:
        comment = re.match("\[%eval (.*)\]", game_state.comment).groups()[0]
    except:
        return None
    board_fen = game_state.board().fen()
    board, turn, castling, enp, halfmove, fullmove = parse_fen(board_fen)
    return fullmove, board_to_nums(board), comment


In [ ]:
import re
re.match("\[%eval (.*)\]", game.end().comment).groups()[0]

## Storing 3d board and evaluation 

In [ ]:
def create_dataset3d(game):
    data1 = []
    data4d= []
    state = game.end()
    while True:
        row = create_row(state)
        if row is not None:
            fullmove, board, comment = row
            board3d = split_dims(board=chess.Board(game.end().board().fen()))
            if comment[0] != "#":
                data1.append(np.append(board, [comment]))
                data4d.append(board3d)
            if comment[0] == "#":
                if comment[1] == "-":
                    data1.append(np.append(board, [-(120 + int(comment[2:])) * 1]))
                    data4d.append(board3d)
                else:
                    data1.append(np.append(board, [(120 - int(comment[1:])) * 1]))
                    data4d.append(board3d)
            if int(fullmove) == 1:
                break
        try:
            state = state.parent
        except:
            break
    return np.asarray(data1),data4d


## adding all 3d boards into array 

In [ ]:
dataset4d = []
dataset=[]
from tqdm import tqdm

for game in tqdm(games):
    game_pgn = StringIO("\n".join(game))
    try:
        game = chess.pgn.read_game(game_pgn)
        data1,data4d = create_dataset3d(game)
        if len(data1.shape) == 2:
            dataset4d.append(data4d)
            dataset.append(data1)
    except:
        continue

In [ ]:
array4dfinal=np.concatenate(dataset4d, axis=0)
array4dfinal.shape

In [ ]:
np.save('trainingdata15.npy', array4dfinal)

In [ ]:
dataset = np.concatenate(dataset)

In [ ]:
np.savetxt("preprocessed15.csv", dataset, delimiter=",", fmt="%s")